# Model soup

In [2]:
import os
os.chdir("/opt/ml/input/level2_semanticsegmentation_cv-level2-cv-10/")

# infer lib
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader

import segmentation_models_pytorch as smp

import albumentations as A
from albumentations.pytorch import ToTensorV2

import yaml
from easydict import EasyDict


def uniform_soup(args, checkpoint_paths, device = "cpu"):
    
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    
    model_module = getattr(smp, args.decoder)
    model = model_module(
        encoder_name=args.encoder,
        encoder_weights=args.encoder_weights,     
        in_channels=3,               
        classes=11,  
        encoder_output_stride=32                   
    )
    model.to(device)
    
    model_dict = model.state_dict()
    soups = {key:[] for key in model_dict}
    checkpoint = {}
    for i, checkpoint_path in enumerate(checkpoint_paths):
        checkpoint = torch.load(os.path.join('./saved2', checkpoint_path), map_location='cpu')
        weight_dict = checkpoint
        for k, v in weight_dict.items():
            soups[k].append(v)
    if 0 < len(soups):
        soups = {k:(torch.sum(torch.stack(v), axis = 0) / len(v)).type(v[0].dtype) for k, v in soups.items() if len(v) != 0}
        model_dict.update(soups)
        model.load_state_dict(model_dict)
    
    return model, checkpoint


################ model cfg path 적기 ################
dataset_path = '/opt/ml/input/data'
CONFIG_FILE_NAME = "./config/config.yaml"
with open(CONFIG_FILE_NAME, "r") as yml_config_file:
    args = yaml.load(yml_config_file, Loader=yaml.FullLoader)
    args = EasyDict(args["train"])

################ soup할 checkpoint path 적기 ################
checkpoint_paths = os.listdir('/opt/ml/input/level2_semanticsegmentation_cv-level2-cv-10/saved2')
################ soup할 checkpoint path 적기 ################
device = "cpu"

################ save dir path 적기 ################
save_dir_path = '/opt/ml/input/level2_semanticsegmentation_cv-level2-cv-10/saved2/'
name = 'soup'
################ save dir path 적기 ################

print("\n[Uniform Soup]")
uniform_model, checkpoint = uniform_soup(args, checkpoint_paths, device = device)
uniform_dict = uniform_model.state_dict()

torch.save(uniform_dict, os.path.join(save_dir_path, f'{name}.pth'))


[Uniform Soup]


AttributeError: 'PAN' object has no attribute 'items'

0. 모델 & checkpoint 가져오기

In [57]:
################ model cfg path 적기 ################
dataset_path = '/opt/ml/input/data'
CONFIG_FILE_NAME = "./config/config.yaml"
with open(CONFIG_FILE_NAME, "r") as yml_config_file:
    args = yaml.load(yml_config_file, Loader=yaml.FullLoader)
    args = EasyDict(args["train"])

################ soup할 checkpoint path 적기 ################
checkpoint_paths = os.listdir('./saved2')
################ soup할 checkpoint path 적기 ################
device = "cpu"

1. uniform soup

In [58]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
    
model_module = getattr(smp, args.decoder)
model = model_module(
        encoder_name=args.encoder,
        encoder_weights=args.encoder_weights,     
        in_channels=3,               
        classes=11,  
        encoder_output_stride=32                   
    )
model.to(device)
    
model_dict = model.state_dict()
soups = {key:[] for key in model_dict}
checkpoint = {}
checkpoint = torch.load(os.path.join('./saved_2', checkpoint_paths[0]), map_location='cpu')
checkpoint2 = torch.load(os.path.join('./saved_2', 'soup.pth'), map_location='cpu')

In [59]:
print(len(checkpoint.keys()))
print(len(checkpoint2.keys()))

457
457


In [60]:
print(set(checkpoint2.keys()) - set(checkpoint.keys()))

set()


In [61]:
print(len(set(checkpoint2.keys()) & set(checkpoint.keys())))

457


In [62]:
checkpoint = torch.load(os.path.join('/opt/ml/input/cv_10/saved_2', 'soup.pth'), map_location='cuda')
model.load_state_dict(checkpoint)

<All keys matched successfully>

잘 되는지 확인

In [63]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from dataloaders.DataLoader import CustomDataLoader
from torchmetrics.classification import MulticlassJaccardIndex
from torchmetrics import MetricCollection

def collate_fn(batch):
    return tuple(zip(*batch))

dataset_path = '/opt/ml/input/data'
val_path = dataset_path + '/kfold/val_fold2.json'

val_transform = A.Compose([
                            ToTensorV2()
                          ])
val_dataset = CustomDataLoader(data_dir=val_path, dataset_path=dataset_path, mode='val', transform=val_transform)

val_loader = DataLoader(dataset=val_dataset, 
                            batch_size=8,
                            shuffle=False,
                            num_workers=4,
                            collate_fn=collate_fn,
                            drop_last=True)

criterion1 = torch.nn.CrossEntropyLoss()

def validation(model, data_loader, device, criterion):
    print(f'Start validation!')
    model.eval()

    with torch.no_grad():
        n_class = 11
        total_loss = 0
        cnt = 0
        
        # metric의 묶음을 한 번에 사용
        metric_collection = MetricCollection({
            "micro": MulticlassJaccardIndex(num_classes=n_class, average="micro"),
            "macro": MulticlassJaccardIndex(num_classes=n_class, average="macro"),      # mIoU
            "classwise": MulticlassJaccardIndex(num_classes=n_class, average="none")    # classwise IoU
        })
        metric_collection.cuda()

        for step, (images, masks, _) in enumerate(tqdm(data_loader)):
            
            images = torch.stack(images)       
            masks = torch.stack(masks).long()  

            images, masks = images.to(device), masks.to(device)            
            
            outputs = model(images)
            loss = criterion(outputs, masks)
            total_loss += loss
            cnt += 1
            
            metric_collection.update(outputs, masks)
        
        result = metric_collection.compute()
        micro = result["micro"].item()
        macro = result["macro"].item()
        classwise_results = result["classwise"].detach().cpu().numpy()
        category_list = ['Background', 'General trash', 'Paper', 'Paper pack', 'Metal',
                'Glass', 'Plastic', 'Styrofoam', 'Plastic bag', 'Battery', 'Clothing']
        IoU_by_class = {classes : round(IoU, 4) for IoU, classes in zip(classwise_results, category_list)}

        avrg_loss = total_loss / cnt
        print(f'Validation || Average Loss: {round(avrg_loss.item(), 4)} || macro : {round(macro, 4)} || micro: {round(micro, 4)}')
        print(f'IoU by class : {IoU_by_class}')
        
    return avrg_loss, macro, IoU_by_class

avrg_loss, val_mIoU, IoU_by_class = validation(model, val_loader, device, criterion1)

loading annotations into memory...
Done (t=0.65s)
creating index...


  0%|          | 0/82 [00:00<?, ?it/s]

index created!
Start validation!


100%|██████████| 82/82 [00:20<00:00,  3.92it/s]

Validation || Average Loss: 0.5695 || macro : 0.2834 || micro: 0.7398
IoU by class : {'Background': 0.8254, 'General trash': 0.4862, 'Paper': 0.6976, 'Paper pack': 0.0003, 'Metal': 0.0, 'Glass': 0.1862, 'Plastic': 0.001, 'Styrofoam': 0.2922, 'Plastic bag': 0.6286, 'Battery': 0.0, 'Clothing': 0.0}
0.2834179401397705
